In [2]:
from collections import deque
import numpy as np
import pandas as pd
import csv
from pulp import *
from pulp import GLPK
import time
import datetime as dt
from datetime import datetime, timedelta
import time
import math

In [3]:
Data = pd.read_csv('exemple.csv', header=0, parse_dates=[0], index_col=0, squeeze=True)
Data.info
Data.dropna()

,Brigade,Terminal,Compagnie,N parking,N vol,Immat,Heure accostage,Heure desaccostage
Date,,,,,,,,
2020-02-23,Journee,T1,AH,W09,6040,JK,8:15,9:10
2020-02-23,Journee,T1,DTH,W12,1903,CB,8:45,10:10
2020-02-23,Journee,T1,DTH,W08,1803,CA,9:55,11:20
2020-02-23,Journee,T1,DTH,W10A,2111,CF,10:05,11:05
2020-02-23,Journee,T1,DTH,W10A,2220,CF,13:50,14:55
2020-02-23,Journee,T1,DTH,W12,2110,CC,14:25,15:15
2020-02-23,Journee,T1,DTH,W10C,1321,CB,14:45,18:10
2020-02-23,Journee,T1,AH,W11A,6006,JU,17:10,18:20
2020-02-23,Journee,T1,AH,W09,6026,KB,17:20,18:25


In [4]:
for i in range (len(Data)):
    try : 
        Data["Heure accostage"][i].day = Data["Date"][i].day
        Data["Heure accostage"][i].month = Data["Date"][i].month
        Data["Heure accostage"][i].year = Data["Date"][i].year
    except: 
        pass
print(Data)


            Brigade Terminal Compagnie N parking  N vol Immat Heure accostage  \
Date                                                                            
2020-02-23  Journee       T1        AH       W09   6040    JK            8:15   
2020-02-23  Journee       T1       DTH       W12   1903    CB            8:45   
2020-02-23  Journee       T1       DTH       W08   1803    CA            9:55   
2020-02-23  Journee       T1       DTH      W10A   2111    CF           10:05   
2020-02-23  Journee       T1       DTH      W10A   2220    CF           13:50   
2020-02-23  Journee       T1       DTH       W12   2110    CC           14:25   
2020-02-23  Journee       T1       DTH      W10C   1321    CB           14:45   
2020-02-23  Journee       T1        AH      W11A   6006    JU           17:10   
2020-02-23  Journee       T1        AH       W09   6026    KB           17:20   
2020-02-23  Journee       T1        AH       W08   6041    JK           17:55   
2020-02-23  Journee       T4

In [5]:

TimeGap=10
def Trans_to_min(x):
    D=x.day*24*60
    min = x.minute
    H = x.hour *60
    return int(D+min+H)
time_Acco= Data["Heure accostage"]
time_Desa= Data["Heure desaccostage"]

Graph_intervale=[]
time=time_Acco
for i in [x for x in range(len(time)) if x != None]:
    # print(time[i])
    # if time[i]== None :
    #     time[i].pop()
    # else:
    try:
        Graph_intervale.append([])
        time[i] = dt.datetime.strptime(str(time[i]), '%Y-%d-%m %H:%M:%S')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max))
    except:
        time[i] = dt.datetime.strptime(str(time[i]), '%H:%M')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[i].append(Trans_to_min(max)) 
time=time_Desa
for i in [x for x in range(len(time)) if x != None]:
    j=i+len(time_Acco)

    try:
        Graph_intervale.append([])
        time[i] = dt.datetime.strptime(str(time[i]), '%Y-%d-%m %H:%M:%S')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[j].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[j].append(Trans_to_min(max))
    except:
        time[i] = dt.datetime.strptime(str(time[i]), '%H:%M')
        min= time[i]- dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[j].append(Trans_to_min(min))
        max= time[i]+ dt.timedelta(minutes=TimeGap//2)
        Graph_intervale[j].append(Trans_to_min(max)) 
print((Graph_intervale))


[[1930, 1940], [1960, 1970], [2030, 2040], [2040, 2050], [2265, 2275], [2300, 2310], [2320, 2330], [2465, 2475], [2475, 2485], [2510, 2520], [1920, 1930], [1931, 1941], [1955, 1965], [2000, 2010], [2105, 2115], [2165, 2175], [2191, 2201], [2270, 2280], [2495, 2505], [1920, 1930], [1935, 1945], [1920, 1930], [1925, 1935], [1935, 1945], [1935, 1945], [1945, 1955], [1970, 1980], [1990, 2000], [2008, 2018], [2028, 2038], [2055, 2065], [2065, 2075], [2075, 2085], [2115, 2125], [2135, 2145], [2228, 2238], [2272, 2282], [2270, 2280], [2290, 2300], [2330, 2340], [2340, 2350], [2370, 2380], [2383, 2393], [2405, 2415], [2425, 2435], [2305, 2315], [2524, 2534], [2514, 2524], [2517, 2527], [2521, 2531], [2543, 2553], [1985, 1995], [2045, 2055], [2115, 2125], [2100, 2110], [2330, 2340], [2350, 2360], [2525, 2535], [2535, 2545], [2540, 2550], [2847, 2857], [1980, 1990], [1994, 2004], [2000, 2010], [2065, 2075], [2141, 2151], [2239, 2249], [2251, 2261], [2335, 2345], [2567, 2577], [1983, 1993], [2003

C:\Users\pc\AppData\Local\Temp/ipykernel_27156/3507195072.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time[i] = dt.datetime.strptime(str(time[i]), '%H:%M')
C:\Users\pc\AppData\Local\Temp/ipykernel_27156/3507195072.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time[i] = dt.datetime.strptime(str(time[i]), '%H:%M')


In [6]:
def Verify_in_interval(M1,M2):
    if M1[0]<=M2[0] and M1[1]>=M2[0]:
        return True
    elif M1[0]<=M2[1] and M1[1]>=M2[1]:
        return True
    else: 
        return False 
def   listAdj(M):
    listAdj=[]
    for i in range(len(M)):
        listAdj.append([])
        for j in [x for x in range(len(M)) if x != i]:
            if Verify_in_interval(M[i],M[j]) or Verify_in_interval(M[j],M[i]) :
                listAdj[i].append(j)
            else:
                pass
    return listAdj
            #print(listAdj[i])
    #print(listAdj[i])
# print(M)
Adj=listAdj(Graph_intervale)
print(Adj)

[[10, 11, 19, 20, 21, 22, 23, 24], [12, 26], [3, 29, 74, 75], [2, 52, 73, 74, 75], [17, 36, 37], [38, 45, 86], [39, 55], [8, 89, 90], [7, 93], [47, 48], [0, 19, 21, 22], [0, 20, 22, 23, 24], [1, 25], [27, 28, 62, 63, 71], [33, 53, 54, 77], [], [], [4, 36, 37], [91, 92], [0, 10, 21, 22], [0, 11, 22, 23, 24, 25], [0, 10, 19, 22], [0, 10, 11, 19, 20, 21, 23, 24], [0, 11, 20, 22, 24, 25], [0, 11, 20, 22, 23, 25], [12, 20, 23, 24], [1, 61], [13, 51, 61, 62, 63, 70, 72], [13, 63, 71], [2, 74, 75], [31, 52, 64, 73, 76, 78], [30, 32, 64, 76], [31, 64, 79], [14, 53, 80], [65, 80, 82], [84], [4, 17, 37], [4, 17, 36], [5, 86], [6, 40, 55, 68], [39, 55, 56, 68], [], [88], [], [96], [5, 86], [47, 48, 49, 57], [9, 46, 48, 49], [9, 46, 47, 49, 57], [46, 47, 48, 57], [58, 59], [27, 61, 62, 70, 72], [3, 30, 73, 78], [14, 33, 80], [14, 77], [6, 39, 40, 68], [40], [46, 48, 49, 58], [50, 57, 59], [50, 58], [], [26, 27, 51, 70, 72], [13, 27, 51, 63, 71, 72], [13, 27, 28, 62, 71], [30, 31, 32, 76], [34], [8

In [7]:
def matriceAdj(Adj):
    Mat=[]
    for i in range( len(Adj)):
        Mat.append([])
        for j in range( len(Adj)):
            Mat[i].append(0)
            for k in range(len(Adj[i])):
                if j==Adj[i][k]:
                    Mat[i][j]=1
    return Mat

In [8]:
def arret(M):
    ex=[[],[]]
    for i in range (len(M)-1):
        for j in range(i+1,len(M)):
            if M[i][j]==1:
                ex[0].append(i)
                ex[1].append(j)
    return ex


In [9]:
Mat=matriceAdj(Adj)
Mat= np.array(Mat)
print (Mat)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
Ex=arret(Mat)
print("ex1= ",Ex[0])
print("ex2= ",Ex[1])


ex1=  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 9, 9, 10, 10, 10, 11, 11, 11, 11, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 18, 18, 19, 19, 20, 20, 20, 20, 21, 22, 22, 23, 23, 24, 26, 27, 27, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 30, 30, 31, 31, 31, 32, 32, 33, 33, 34, 34, 34, 35, 36, 38, 39, 39, 39, 40, 40, 40, 42, 44, 45, 46, 46, 46, 46, 47, 47, 48, 48, 49, 50, 50, 51, 51, 51, 51, 52, 52, 53, 54, 55, 57, 58, 61, 61, 62, 62, 62, 63, 64, 66, 69, 69, 70, 73, 73, 74, 76, 80, 83, 89, 90, 91, 92, 95, 97]
ex2=  [10, 11, 19, 20, 21, 22, 23, 24, 12, 26, 3, 29, 74, 75, 52, 73, 74, 75, 17, 36, 37, 38, 45, 86, 39, 55, 8, 89, 90, 93, 47, 48, 19, 21, 22, 20, 22, 23, 24, 25, 27, 28, 62, 63, 71, 33, 53, 54, 77, 36, 37, 91, 92, 21, 22, 22, 23, 24, 25, 22, 23, 24, 24, 25, 25, 61, 51, 61, 62, 63, 70, 72, 63, 71, 74, 75, 31, 52, 64, 73, 76, 78, 32, 64, 76, 64, 79, 53, 80, 65, 80, 82, 84, 37, 86, 40, 55, 68, 55, 56, 68, 88, 96, 86, 47, 48, 

In [11]:
def Gloton(Adj):
    n=len(Adj)
    NC=dict()
    C=[0 for k in range(n)]
    for i in range(n):
        NC[i]=[]
    def  rajouter(k,NC):
        if ((k in NC)==True):
            return NC
        else:
            NC.append(k)
            return NC
    i=0
    while i<=n-1:
        exist=True
        k=0
        while (exist==True):
            exist=k in NC[i]
            if (exist==True):
                k+=1
        C[i]=k    
        for j in range(len(Adj[i])):
            voisin=Adj[i][j]
            NC[voisin]=rajouter(k,NC[voisin])
        i+=1
    return C
C=Gloton(Adj)
print(C)

[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 2, 2, 3, 4, 3, 5, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 2, 3, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 2, 3, 0, 0, 2, 2, 1, 2, 1, 1, 2, 1, 0, 2, 2, 3, 2, 1, 0, 0, 3, 0, 3, 4, 4, 3, 2, 3, 4, 2, 1, 1, 3, 0, 1, 0, 1, 1, 2, 0, 1, 1, 2, 1, 2, 0, 0, 1, 1, 0, 0, 0, 2, 1]


In [12]:
# %%
# Import of the pyomo module
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pyomo.environ

# Creation of a Concrete Model
model = ConcreteModel()

# %%
model.C = Set(initialize=range(6), doc='les Conducteur')
model.J = Set(initialize=range(len(Adj)), doc='les jobs')
model.A = Set(initialize=range(len(Ex[1])), doc='les arcs')

# %%
model.n = Param(initialize=len(Adj), doc='le nombre de jobs')
model.c = Param(initialize=6, doc='le nombre de conducteur')
model.m = Param(initialize=len(Ex[1]), doc='les relations entre les arcs')
model.ex1 = Param(model.A,initialize=Ex[0], doc='extrémité 1')
model.ex2 = Param(model.A,initialize=Ex[1], doc='extrémité 2')

# %%
model.x = Var(model.C, model.J,domain=Binary, doc='affectation conducteur')
model.v = Var(model.C,bounds=(0,None),domain=Integers, doc='les taches')
model.y = Var(bounds=(0,None),domain=Integers, doc='valeur de comparaison')

# %%
def Cotraint1(model,j):
  return sum(model.x[i,j] for i in model.C) == 1
model.C1 = Constraint(model.J, rule=Cotraint1, doc='verification daffectation')


def Contraint2(model,i,k):
  return ((model.x[i,model.ex1[k]]+model.x[i,model.ex2[k]])) <= 1  

model.C2 = Constraint (model.C,model.J, rule=Contraint2, doc='verification de non chevauchement ')

def Contraint3(model,i):
  return sum(model.x[i,j] for j in model.J)==model.v[i]

model.C3 = Constraint (model.C, rule=Contraint3, doc='une seul tache par conducteur')

def Contraint4(model,i):
  return (model.v[i]<=model.y)

model.C4 = Constraint (model.C, rule=Contraint4, doc='maximiser')

# %%
def objective_rule(model):
  return model.y
model.objective = Objective(rule=objective_rule, sense=minimize, doc='Define objective function')

# %%
def pyomo_postprocess(options=None, instance=None, results=None):
  model.x.display()
# %%


opt = SolverFactory("gurobi")
results = opt.solve(model,tee=True)
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)
# %%




Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-10
Read LP format model from file C:\Users\pc\AppData\Local\Temp\tmpq5mtxvuq.pyomo.lp
Reading time = 0.01 seconds
x620: 727 rows, 620 columns, 2467 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 727 rows, 620 columns and 2467 nonzeros
Model fingerprint: 0x316aa9cf
Variable types: 1 continuous, 619 integer (612 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 20.0000000
Presolve removed 307 rows and 7 columns
Presolve time: 0.02s
Presolved: 420 rows, 613 columns, 2016 nonzeros
Variable types: 0 continuous, 613 integer (612 binary)

Root relaxation: objective 1.700000e+01, 210 iterations, 0.01 seconds (0.00 work units)

    Nodes    |

In [13]:
x=model.x
Group_co=[]
for i in model.C:
    Group_co.append([])
    for j in model.J:
        if x[i,j].value == 1:
            Group_co[i].append(j)

print(Group_co)
for i in range(len(Group_co)):
    print("list des vols du conducteur "+str(i+1))
    print("Terminal       N Parking          Time")
    for j in Group_co[i]:
        if j<len(time_Acco):
            print(Data["Terminal"][j]+'  |  '+Data["N parking"][j]+'  |  '+str(time_Acco[j])+"  |   Heure Accostage")
        else:
            j=j-len(time_Acco)
            print(Data["Terminal"][j]+'  |  '+Data["N parking"][j]+'  |  '+str(time_Desa[j])+"  |   Heure Desacostage")



[[3, 6, 9, 10, 13, 20, 40, 43, 44, 45, 50, 53, 79, 92, 98, 99, 100], [1, 8, 14, 17, 18, 19, 24, 27, 28, 29, 30, 34, 35, 39, 41, 83, 90], [5, 7, 23, 37, 42, 46, 48, 51, 54, 60, 62, 75, 78, 81, 87, 91, 101], [4, 11, 12, 21, 52, 56, 58, 63, 64, 70, 71, 73, 80, 82, 84, 94, 95], [2, 22, 32, 36, 38, 49, 55, 61, 65, 66, 68, 76, 85, 88, 89, 96, 97], [0, 15, 16, 25, 26, 31, 33, 47, 57, 59, 67, 69, 72, 74, 77, 86, 93]]
list des vols du conducteur 1
Terminal       N Parking          Time
T1  |  W10A  |  1900-01-01 10:05:00  |   Heure Accostage
T1  |  W10C  |  1900-01-01 14:45:00  |   Heure Accostage
T1  |  W08  |  1900-01-01 17:55:00  |   Heure Accostage
T4  |  W29C  |  1900-01-01 08:05:00  |   Heure Accostage
T4  |  W14  |  1900-01-01 09:25:00  |   Heure Accostage
T4  |  W22A  |  1900-01-01 08:20:00  |   Heure Accostage
T4  |  W17  |  1900-01-01 15:05:00  |   Heure Accostage
T4  |  W23A  |  1900-01-01 16:10:00  |   Heure Accostage
T4  |  W14  |  1900-01-01 16:30:00  |   Heure Accostage
T4  |  W1